In [1]:
# DIVIDE AND CONQUER
# This script takes important reactions identified by divide and conquer and generates cti models
# that try different variations on those reactions

In [18]:
# use sobol randoms to pick which reaction kinetics to use
# Generate the random numbers to do the mixing
important_reactions = [0, 541, 262,  26, 270, 804, 157, 355, 369, 412, 496, 260]

N = 30
assert N == 30  # you'll run into reproducibility issues if you keep changing this up
num_flame_speeds = 2 ** 11
# num_flame_speeds = 2 ** 5
sampler = scipy.stats.qmc.Sobol(d=N, scramble=True, seed=400)
sample = sampler.random(num_flame_speeds)
binary_sample = sample > 0.5
binary_sample.shape

(2048, 30)

In [22]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
import numpy as np
import subprocess

import csv

import mix_thermokinetics

import scipy.stats
import copy
import os

In [2]:
# Load the models
# load David's 2-BTP model, minus the halogens
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

RMG_chemkin_path = 'models/RMG_no_halogens/no_halogens.inp'
RMG_dictionary_path = 'models/RMG_no_halogens/no_halogens_dict.txt'
RMG_transport_path = 'models/RMG_no_halogens/no_halogens_tran.dat'
RMG_cti_path = 'models/RMG_no_halogens/no_halogens.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)


# Load NIST model without halogens
NIST_cti_path = 'models/NIST_no_halogens/no_halogens.cti'
NIST_chemkin_path = 'models/NIST_no_halogens/no_halogens.inp'
NIST_dictionary_path = 'models/NIST_no_halogens/no_halogens_dict.txt'
NIST_transport_path = 'models/NIST_no_halogens/no_halogens_tran.dat'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path, transport_path=NIST_transport_path)



For species CH2OCH, discontinuity in cp/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  8.393471510000001
	Value computed using high-temperature polynomial: 9.1801039121875

For species CH2OCH, discontinuity in h/RT detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  42.199147089791666
	Value computed using high-temperature polynomial: 41.961461604875005

For species CH2OCH, discontinuity in s/R detected at Tmid = 500.0
	Value computed using low-temperature polynomial:  33.70692865946735
	Value computed using high-temperature polynomial: 33.51209988778391

For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  47.65235236593109
	Value computed using high-temperature polynomial: 48.43623165666667

For species C4H5-2, discontinuity in s/R detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  52.42918829260522
	Value computed using high-temperature p

In [4]:
# get the mapping between RMG and NIST models
# Species Diff
common_species = []
RMG2NIST_mapping = {}
NIST2RMG_mapping = {}
for i, rmg_sp in enumerate(RMG_species_list):
    for j, nist_sp in enumerate(NIST_species_list):
        if rmg_sp.is_isomorphic(nist_sp):
            RMG2NIST_mapping[i] = j
            NIST2RMG_mapping[j] = i
            common_species.append([rmg_sp, nist_sp])
            break

# Reaction Diff
common_reactions = []
RMG2NIST_rxn_mapping = {}
NIST2RMG_rxn_mapping = {}
for i, rmg_rxn in enumerate(RMG_reaction_list):
    for j, nist_rxn in enumerate(NIST_reaction_list):
        if rmg_rxn.is_isomorphic(nist_rxn):
            RMG2NIST_rxn_mapping[i] = j
            NIST2RMG_rxn_mapping[j] = i
            common_reactions.append([rmg_rxn, nist_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

45 common species
240 common reactions


In [5]:
# Convert the NIST species in the reactions to RMG species, but keep the NIST kinetics
def NIST2RMG(nist_reaction):
    rmg_reaction = copy.deepcopy(nist_reaction)
    reactants = []
    for reactant in nist_reaction.reactants:
        try:
            NIST_species_index = NIST_species_list.index(reactant)
            reactants.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if reactant in RMG_species_list:
                reactants.append(reactant)
        
    rmg_reaction.reactants = reactants
    
    products = []
    for product in nist_reaction.products:
        try:
            NIST_species_index = NIST_species_list.index(product)
            products.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if product in RMG_species_list:
                products.append(product)
    rmg_reaction.products = products
    
    return rmg_reaction


In [6]:
def RMG2NIST(RMG_reaction):
    # takes in the RMG_reaction object to convert
    RMG_index = RMG_reaction_list.index(RMG_reaction)
    if RMG_index not in RMG2NIST_rxn_mapping.keys():
        # this reaction does not exist in NIST, so it will be deleted. return None
        return
    NIST_index = RMG2NIST_rxn_mapping[RMG_index]
    NIST_reaction = NIST_reaction_list[NIST_index]
    
    # convert the NIST model species in the NIST_reaction to RMG model species
    return NIST2RMG(NIST_reaction)

In [19]:
divide_model_dir = 'divide_models'
os.makedirs(divide_model_dir, exist_ok=True)

In [21]:
# Generate the models
for k in range(0, 2):
    # convert the indicated reactions to use the NIST kinetics
    new_reaction_list = []
    deleted_duplicates = []
    for i in range(0, len(RMG_reaction_list)):
        if i in important_reactions and binary_sample[k, important_reactions.index(i)]:
            new_reaction = RMG2NIST(RMG_reaction_list[i])
            if new_reaction:
                new_reaction_list.append(new_reaction)
            elif RMG_reaction_list[i].duplicate:
                deleted_duplicates.append(RMG_reaction_list[i])
        else:
            new_reaction_list.append(RMG_reaction_list[i])
            
            
    # get rid of duplicates
    for i, rxn in enumerate(new_reaction_list):
        if rxn.duplicate:
            duplicate_still_exists = False
            for j, rxn2 in enumerate(new_reaction_list):
                if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                    duplicate_still_exists = True
                    break
            if not duplicate_still_exists:
                rxn.duplicate = False

    # mark reactions that are duplicates
    for i, rxn in enumerate(new_reaction_list):
        if not rxn.duplicate:
            duplicate = False
            for j, rxn2 in enumerate(new_reaction_list):
                if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                    duplicate = True
                    break
            if duplicate:
                rxn.duplicate = True
                
                
    # save the models
    chemkin_file = os.path.join(divide_model_dir, f'chem_{k:04}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, RMG_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)
    subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={RMG_transport_path}', f'--output={divide_model_dir}/chem_{k:04}.cti'])
    os.remove(chemkin_file)

Wrote CTI mechanism file to 'divide_models/chem_0000.cti'.
Mechanism contains 98 species and 828 reactions.
Validating mechanism...PASSED.
Wrote CTI mechanism file to 'divide_models/chem_0001.cti'.
Mechanism contains 98 species and 827 reactions.
Validating mechanism...PASSED.


Wrote CTI mechanism file to 'divide/chem_0.cti'.
Mechanism contains 98 species and 823 reactions.
Validating mechanism...PASSED.


# Run Flame Speeds

In [131]:
def run_flame_speed(cti_file, output_file='flame_speed.csv'):
    gas = ct.Solution(cti_file)
    
    T0 = 298
    P0 = ct.one_atm
    equiv_ratios = [1.0]

    # can use below to only test one vol_frac
    vol_frac_list = [0.0]

    # Define stoichiometric coefficients for methane combustion reaction
    v_fuel = 1.0
    v_oxidizer = 2.0
    results = {}

    for i, phi in enumerate (equiv_ratios):
        try:

            tol_ss = [1.0e-13, 1.0e-9]  # abs and rel tolerances for steady state problem
            tol_ts = [1.0e-13, 1.0e-9]  # abs and rel tie tolerances for time step function

            # calculate composition from equivalence ratio
            actual_ratio = phi * (v_fuel / v_oxidizer)
            x_O2 = 1.0
            x_CH4 = actual_ratio * x_O2
            x_N2 = 0.79 * (x_O2 / .21)
            total = x_O2 + x_CH4 + x_N2
            x_O2 = x_O2 / total
            x_CH4 = x_CH4 / total
            x_N2 = x_N2 / total
            print(f'Starting compositions: x_O2={x_O2}, x_CH4={x_CH4}, x_N2={x_N2}')
            mol_fractions = {'CH4(3)': x_CH4, 'O2(4)': x_O2, 'N2': x_N2}

            # Define starting concentrations in molar ratios
            gas.TPX = T0, P0, mol_fractions
            width = 0.08
            flame = ct.FreeFlame(gas, width=width)
            flame.flame.set_steady_tolerances(default=tol_ss)   # set tolerances
            flame.flame.set_transient_tolerances(default=tol_ts)
            flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
            flame.max_time_step_count = 900
            loglevel = 1

            print("about to solve")
            flame.solve(loglevel=loglevel, auto=True)
            Su = flame.velocity[0]
            results[phi] = Su
            sltn = flame.to_solution_array()
            df1 = sltn.to_pandas()
            # edited this here!! index=False

        except Exception as e:
            print(f'********************passed volume fraction:{equiv_ratios[i]}, error: {e}*************************************')
            pass


    equiv_ratios = list(results.keys())
    flame_speeds = list(results.values())

    print("equivalence ratios are:")
    print(equiv_ratios)

    print("flame speeds are:")
    print(flame_speeds)
    
    with open(output_file, 'w+') as g:
        writers = csv.writer(g)
        writers.writerow(equiv_ratios)
        writers.writerow(flame_speeds)
    
    return flame


In [155]:
run_flame_speed('divide/chem_0.cti', 'divide/flame_speed_0.csv')

Starting compositions: x_O2=0.19004524886877827, x_CH4=0.09502262443438914, x_N2=0.7149321266968326
about to solve

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      3.914
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.002189      4.156
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     3.653e-05      5.855
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.734e-05      6.055
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0004444      4.732
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
grid refi

Attempt Newton solution of steady-state problem...    success.

Problem solved on [58] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 
    to resolve C(T)(17) C2H(23) C2H2(24) C2H3(30) C2H4(31) C2H5(33) C2H6(32) C3H3(4828) C3H4(2596) C3H4(5216) CH(18) CH2(S)(26) CH2(T)(19) CH2CHO(36) CH2CO(29) CH2O2(67) CH2OH(34) CH3(20) CH3CHO(37) CH3O(28) CH3OO(201) CHCHOH(59) CHO3(90) H2CC(25) H2O2(13) HCCO(22) HCO(16) S(105) S(26591) S(28060) 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [75] point grid(s).

...........................................................

In [156]:
run_flame_speed('divide/chem_1.cti', 'divide/flame_speed_1.csv')

Starting compositions: x_O2=0.19004524886877827, x_CH4=0.09502262443438914, x_N2=0.7149321266968326
about to solve

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      3.449
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003649      4.856
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.827e-05      6.825
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     6.935e-05      5.451
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.001777      3.521
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.04555      2.829
Attempt Newton solution of steady-state problem...    success.

Problem solved on [

Attempt Newton solution of steady-state problem...    success.

Problem solved on [60] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 
    to resolve C2H(23) C2H2(24) C2H3(30) C2H4(31) C3H4(2596) C3H4(5216) CH(18) CH2(S)(26) CH2(T)(19) CH2CHO(36) CH2CO(29) CH2OH(34) CH3(20) CH3CHO(37) CH3O(28) CH3OO(201) CHO3(90) H2CC(25) HCCO(22) HCO(16) S(105) S(26591) 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [77] point grid(s).

..............................................................................
#########################################################

In [162]:
# list the reactions highlighted
reactions_to_use_NIST
for i in range(0, len(reactions_to_use_NIST)):
    print(RMG_reaction_list[reactions_to_use_NIST[i]])

O2(4) + H(5) <=> O(6) + OH(7)
HCO(16) + C3H3(4828) <=> CO(14) + propyne(2596)
H(5) + C(T)(17) <=> CH(18)
O2(4) + CO(14) <=> O(6) + CO2(15)
OH(7) + CH2(T)(19) <=> CH2OH(34)
O2(4) + [CH]=C=C=C=O(47221) <=> [O]OC=C=C=C=O(51137)
CH3OH(27) + C2H3(30) <=> CH3O(28) + C2H4(31)
HO2(12) + CH3O(28) <=> O2(4) + CH3OH(27)
C2H3(30) + C2H5(33) <=> C2H4(31) + C2H4(31)
O(6) + CH2O(21) <=> [CH2]O[O](67)
O(6) + CH3CO(35) <=> acetyloxy(141)
O(6) + OH(7) <=> HO2(12)


In [163]:
reactions_to_use_NIST

array([  0, 541, 262,  26, 270, 804, 157, 355, 369, 412, 496, 260])